# ComfyUI for Google Colab - 教育用完全版

**🎨 ワンクリックでComfyUIが起動する魔法のノートブック**

## 🚀 使用方法
1. 「ランタイム」→「ランタイムのタイプを変更」→「T4 GPU」を選択
2. 下のセルを実行（約10-15分）
3. 表示される `https://xxxxx.trycloudflare.com` をクリック
4. ComfyUIで画像生成開始！

## 📁 保存先
生成画像は `/content/drive/MyDrive/ComfyUI/output/` に永続保存されます。

In [ ]:
from google.colab import drive
import os, sys, time, re, socket, subprocess, threading
from pathlib import Path

print('🎨 ComfyUI Google Colab 教育版セットアップ')
print('=' * 50)

# Google Colab専用設定
COMFY_DIR = Path('/content/drive/MyDrive/ComfyUI')
MGR_DIR = COMFY_DIR / 'custom_nodes' / 'ComfyUI-Manager'
CHECKPOINT_DIR = COMFY_DIR / 'models' / 'checkpoints'
OUTPUT_DIR = COMFY_DIR / 'output'
PORT = 8123

def run_cmd(cmd, check=True, shell=False, cwd=None, quiet=False):
    if not quiet: print('$ ' + (cmd if isinstance(cmd, str) else ' '.join(cmd)))
    return subprocess.run(cmd, check=check, shell=shell, cwd=cwd, capture_output=quiet)

print('\n📁 Step-1: Googleドライブをマウントします...')
drive.mount('/content/drive', force_remount=False)

print('\n🧹 Step-2: 環境をクリーンアップします...')
subprocess.run('pkill -f cloudflared || true', shell=True)
subprocess.run('pkill -f "python.*main.py" || true', shell=True)
subprocess.run(f'fuser -k {PORT}/tcp || true', shell=True)
time.sleep(2)

print('\n📥 Step-3: ComfyUIとManagerをセットアップします...')
# 必要ディレクトリ作成
for directory in [CHECKPOINT_DIR, OUTPUT_DIR, COMFY_DIR / 'input']:
    directory.mkdir(parents=True, exist_ok=True)

# ComfyUI本体
if COMFY_DIR.exists() and (COMFY_DIR / 'main.py').exists():
    try:
        run_cmd(['git', 'pull', '--rebase', '--autostash'], cwd=str(COMFY_DIR), check=False, quiet=True)
        print('🔄 既存ComfyUIを更新しました')
    except:
        print('🔄 更新をスキップします')
else:
    if COMFY_DIR.exists():
        run_cmd(['rm', '-rf', str(COMFY_DIR)], check=False, quiet=True)
    print('📥 ComfyUIを新規インストール中...')
    run_cmd(['git', 'clone', '--depth=1', 'https://github.com/comfyanonymous/ComfyUI.git', str(COMFY_DIR)])

# ComfyUI-Manager
if not MGR_DIR.exists():
    print('📥 ComfyUI-Manager追加中...')
    run_cmd(['git', 'clone', '--depth=1', 'https://github.com/ltdrdata/ComfyUI-Manager.git', str(MGR_DIR)])

print('\n📦 Step-4: 依存関係をインストールします...')
run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '--upgrade', 'pip', 'setuptools', 'wheel'], quiet=True)

# PyTorch確認
try:
    import torch
    torch_ver = torch.__version__.split('+')[0]
    print(f'✅ PyTorch {torch_ver} 検出済み')
except ImportError:
    print('📥 PyTorchインストール中...')
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', 'torch', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu121'], quiet=True)
    import torch
    torch_ver = torch.__version__.split('+')[0]

# ComfyUI要件
req_file = COMFY_DIR / 'requirements.txt'
if req_file.exists():
    print('🔧 ComfyUI依存関係インストール中...')
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(req_file), '--prefer-binary'], check=False, quiet=True)

# Manager要件（エラー許容）
mgr_req = MGR_DIR / 'requirements.txt'
if mgr_req.exists():
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(mgr_req)], check=False, quiet=True)

# xformers自動整合
try:
    major_minor = '.'.join(torch_ver.split('.')[:2])
    xformers_map = {'2.0': '0.0.20', '2.1': '0.0.22.post7', '2.2': '0.0.25', 
                   '2.3': '0.0.27', '2.4': '0.0.28.post1', '2.5': '0.0.28.post1'}
    xformers_ver = xformers_map.get(major_minor, '0.0.28.post1')
    print(f'🔧 xformers {xformers_ver} 整合中...')
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', f'xformers=={xformers_ver}'], check=False, quiet=True)
except:
    pass

print('\n🤖 Step-5: 基本モデル準備（オプション）...')
model_path = CHECKPOINT_DIR / 'v1-5-pruned-emaonly.safetensors'
if not model_path.exists():
    model_url = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors'
    HF_TOKEN = os.environ.get('HUGGINGFACE_TOKEN', '').strip()
    
    print('📥 SD1.5ダウンロード試行中...')
    download_success = False
    
    if HF_TOKEN:
        try:
            cmd = f'wget -q --timeout=300 -O "{model_path}" --header="Authorization: Bearer {HF_TOKEN}" "{model_url}"'
            result = subprocess.run(cmd, shell=True, timeout=600)
            if result.returncode == 0 and model_path.exists():
                download_success = True
                print('✅ 認証ダウンロード完了')
        except:
            pass
    
    if not download_success:
        try:
            result = subprocess.run(['wget', '--timeout=300', '-c', model_url, '-O', str(model_path)], timeout=600, check=False)
            if result.returncode == 0 and model_path.exists():
                download_success = True
                print('✅ モデルダウンロード完了')
        except:
            pass
    
    if not download_success:
        print('💡 モデルダウンロード失敗（後でManagerから追加可能）')
else:
    print('✅ 基本モデル存在確認')

print('\n🌐 Step-6: cloudflaredを準備します...')
if not Path('/usr/local/bin/cloudflared').exists():
    run_cmd(['wget', '-q', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64', '-O', '/usr/local/bin/cloudflared'], quiet=True)
    run_cmd(['chmod', '+x', '/usr/local/bin/cloudflared'], quiet=True)

def wait_for_port(host, port, timeout=180):
    print(f'🔍 ComfyUI起動を待機中...')
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=2):
                print('🎉 ComfyUIが起動しました！')
                return True
        except:
            time.sleep(2)
    print('❌ ComfyUI起動タイムアウト')
    return False

def start_comfyui():
    print('\n🚀 ComfyUIを起動します...')
    env = os.environ.copy()
    env['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
    
    cmd = [
        sys.executable, 'main.py',
        '--listen=0.0.0.0',
        f'--port={PORT}',
        f'--output-directory={OUTPUT_DIR}',
        '--disable-cuda-malloc',
        '--normalvram'
    ]
    
    proc = subprocess.Popen(cmd, cwd=str(COMFY_DIR), env=env,
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                           text=True, bufsize=1)
    
    startup_keywords = ['Uvicorn running', 'Application startup complete']
    for line in iter(proc.stdout.readline, ''):
        if not line: break
        print(f'[ComfyUI] {line.strip()}')
        if any(keyword in line for keyword in startup_keywords):
            print('🎊 ComfyUI完全起動完了！')
            break
    return proc

def start_cloudflared():
    print('\n🌐 外部アクセス用トンネルを作成中...')
    cmd = ['cloudflared', 'tunnel', f'--url=http://0.0.0.0:{PORT}',
           '--protocol', 'http2', '--no-autoupdate', '--loglevel', 'warn']
    
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                           text=True, bufsize=1)
    
    for line in iter(proc.stdout.readline, ''):
        if not line: break
        print(f'[cloudflared] {line.strip()}')
        
        url_match = re.search(r'https://[a-z0-9-]+\\.trycloudflare\\.com', line)
        if url_match:
            public_url = url_match.group(0)
            print('\n' + '🎉' * 20)
            print(f'✨ 魔法のURL: {public_url}')
            print('📱 上記URLをクリックしてComfyUIにアクセス！')
            print('🎉' * 20)
            break
    return proc

print('\n' + '=' * 50)
print('🎯 ComfyUI起動シーケンス開始')
print('=' * 50)

try:
    # ComfyUIをバックグラウンドで起動
    comfy_thread = threading.Thread(target=start_comfyui, daemon=True)
    comfy_thread.start()
    
    # ポート開通待機
    if wait_for_port('127.0.0.1', PORT):
        # cloudflaredトンネル作成
        cf_proc = start_cloudflared()
        
        print('\n🎊 ComfyUI教育環境セットアップ完了！')
        print(f'📁 生成画像保存先: {OUTPUT_DIR}')
        
        # セッション維持
        try:
            while True:
                time.sleep(30)
                if cf_proc.poll() is not None:
                    print('⚠️ トンネル接続が切断されました')
                    break
        except KeyboardInterrupt:
            print('\n🛑 セッション終了要求を受けました')
    else:
        print('❌ ComfyUI起動に失敗しました')

except Exception as e:
    print(f'❌ 予期しないエラー: {e}')
    print('💡 ランタイムを再起動してください')